# RAG: Chat with a PDF

Author: Maria Lentini

Description: Setting Up and Testing "Chat with a PDF" for Retrieval-Augmented Generation (RAG) application with Ollama.

Relevant links:

- RAG tutorial: https://docs.openwebui.com/tutorials/tips/rag-tutorial/

- Ollama github: https://github.com/ollama/ollama

- Ollama official Docker image: https://hub.docker.com/r/ollama/ollama

- Ollama model library: https://ollama.com/library


In [1]:
import sys
from tika import parser # pip install tika

if '../src' not in sys.path:
    sys.path.append('../src')

# import custom module from src/
from pdfChatBot import pdfChatBot

In [7]:
%load_ext autoreload
%autoreload 2

### 1. Create ElasticSearch Index

First run ``docker compose up`` to spin up ElasticSearch and Ollama containers. Initializing pdfChatBot() does two things:

1. Creates an ElasticSearch index with `index_name` 
2. Downloads the model with `model_name` into the Ollama container

In [4]:
# init module
bot = pdfChatBot(index_name='pdf-chat-bot', model_name='gemma:2b')

Failed to create index: {'error': {'root_cause': [{'type': 'resource_already_exists_exception', 'reason': 'index [pdf-chat-bot/T8HhWhQHRduaOiBb-6-uYg] already exists', 'index_uuid': 'T8HhWhQHRduaOiBb-6-uYg', 'index': 'pdf-chat-bot'}], 'type': 'resource_already_exists_exception', 'reason': 'index [pdf-chat-bot/T8HhWhQHRduaOiBb-6-uYg] already exists', 'index_uuid': 'T8HhWhQHRduaOiBb-6-uYg', 'index': 'pdf-chat-bot'}, 'status': 400}
Output: 
Error: ⠋ ⠙ ⠹ ⠸ ⠴ ⠴ ⠦ ⠧ ⠇ ⠏ ⠙ ⠹ ⠸ ⠸ ⠼ ⠴ ⠧ ⠧ ⠇ ⠋ ⠋ ⠹ ⠹ ⠼ ⠴ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠼ ⠼ ⠴ ⠧ ⠧ ⠇ ⠋ ⠙ ⠙ ⠹ ⠼ ⠼ ⠦ 


Ensure that text embedding is working. Get embedding dimension.

In [8]:
# create a sample embedding and verify it's length
# embedding length must match in elastic search
embedding = bot.generate_embedding('The weather is fine today.')
print(len(embedding))

2048


Check that index exists.

In [12]:
# check that index exists
bot.check_index_exists()

True

### 2. Read and Chunk PDF

Parse PDF and extract raw text content.

In [71]:
# read pdf
raw = parser.from_file('../data/k9_policy_2002.pdf')

# extract content
pdf_text = raw['content']

Check out a chunk of text. Each chunk will be vectorized using the Ollama model.

In [103]:
chunks = bot.semantically_break_text(pdf_text)

print(f'Count of chunks: {len(chunks)}\n')
print(chunks[0])

Count of chunks: 101

7/20021  K-9 TRAINING   K-9 Training Standards and Qualification Requirements for New Jersey Law Enforcement  Issued December 1992 Revised July 1995 Revised July 2002  INTRODUCTION  In April 1992, an Advisory Group was established by the Attorney General to establish a statewide standard for training K-9 teams, that is, police officer handler- police dog teams, utilized in New Jersey law enforcement. This group, comprised of  K-9 officers and representatives from various types of law enforcement agencies throughout New Jersey, was to examine relevant training issues and suggest recommendations for a uniform, statewide training standard.


### 3. Index PDF

Break the PDF up into paragraph, iterate each paragraph using the Ollama model to create an embeddings, and creating and index for that embedding in ElasticSearch.

In [102]:
bot.index_pdf_paragraphs(pdf_text)

100%|██████████| 101/101 [04:26<00:00,  2.64s/it]


Fetch a document from the index to confirm that indexing worked.

In [104]:
# Fetch a sample document from the index to verify it's there
response = bot.fetch_document_from_es()

print('Count of hits:', len(response['hits']))

Count of hits: 3


### 4. Perform RAG

Use input text to perform a vector search using ElasticSearch, retreiving the top 5 embeddings, and setting the corresponding text as context for the subsequent Ollama chat request.

Perform search and format request:

In [105]:
# From PDF:
# "In April 1992, an Advisory Group was established by the Attorney General to
# establish a statewide standard for training K-9 teams..."
query = "What happened in April 1992?"

# get top 5 hits from es
hits = bot.search_similar_paragraphs(query)

# format and print context
context = bot.format_context(hits)
print(context)

Top 5 relevant documents:

Document 1:
2.5.1 The police patrol dog, on command from the police officer handler, will demonstrate the ability to remain in a guard position while the police officer handler searches or questions a "suspect."  2.5.2 When the safety of the police officer handler is threatened, the police patrol dog (without command) will demonstrate the ability to physically apprehend a "suspect" until the "suspect" is taken into custody (and a release command is issued).

Document 2:
1.3.5 The police officer handler will identify the general types of information to be included in a departmental K-9 policy, including:  � the circumstances or conditions under which K-9 teams may and may not be utilized;   � the deployment and use of K-9 teams and services;  � the role and responsibilities of the police officer handler, supervisory personnel and other officers;   � reporting requirements and record keeping;   � the training, qualification and re-evaluation of K-9 teams; and  

### Define Questions for Chat Bot

In [119]:
query_1 = "What is the purpose of a police patrol dog?"
query_2 = "What abilities must the police patrol dog exhibit?"
query_3 = "What is the A K-9 team comprised of?"
query_4 = "Describe the basic training for K-9 patrol team officers."
query_5 = "What should be included in in-service training and re-evaluation records?"

### Questions with RAG

#### Question 1 - with RAG

In [115]:
# get full response from chat bot
full_response_text = bot.generate_response(query_1)

print(query_1, '\n')
print(full_response_text)

What is the purpose of a police patrol dog? 

According to Document 1, the purpose of a police patrol dog is to demonstrate the ability to properly search, find, and indicate or retrieve a variety of articles with a human scent (such as clothing, a gun, a wallet, or a screwdriver) within a specified area, including buildings and interior structures and extended, exterior areas of various terrains.


#### Question 2 - with RAG

In [108]:
# get full response from chat bot
full_response_text = bot.generate_response(query_2)

print(query_2, '\n')
print(full_response_text)

The police patrol dog must exhibit the following abilities:

- The ability to properly search, find, and indicate or retrieve a variety of articles with a human scent (such as clothing, a gun, a wallet, or a screwdriver) within a specified area, including buildings and interior structures and extended, exterior areas of various terrains.

- The ability to respond to various commands while walking at the police officer handler's side.

- The ability to properly control the police specialty dog during searches.

- The ability to conduct proper searches to locate a "suspect," "subject," or "evidence" within buildings, interior structures and extended, exterior areas of various terrains.


#### Question 3 - with RAG

In [110]:
# get full response from chat bot
full_response_text = bot.generate_response(query_3)

print(query_3, '\n')
print(full_response_text)

The A K-9 team is comprised of the police officer handler and the police dog.


#### Question 4 - with RAG

In [113]:
# get full response from chat bot
full_response_text = bot.generate_response(query_4)

print(query_4, '\n')
print(full_response_text)

The passage describes the basic training for K-9 patrol teams in New Jersey Law Enforcement. The training encompasses police dog obedience, agility, scent work, criminal apprehension and handler protection, and socialization.


#### Question 5 - with RAG

In [114]:
# get full response from chat bot
full_response_text = bot.generate_response(query_5)

print(query_5, '\n')
print(full_response_text)

What should be included in in-service training and re-evaluation records? 

According to the documents, in-service training and re-evaluation records should include the following information:

- Content of training or re-evaluation program
- Who participated in the training or re-evaluation
- When and where the training or re-evaluation took place
- Instructor's name and credentials
- Assessment results and outcomes
- Goals and objectives achieved
- Accomplishments and certifications obtained


### Questions without RAG
#### Question 1 - No RAG

In [117]:
full_response_text = bot.generate_response_without_rag(query_1)

print(query_1, '\n')
print(full_response_text)

What is the purpose of a police patrol dog? 

**Purpose of a Police Patrol Dog**

A police patrol dog serves multiple important purposes for law enforcement agencies, including:

**1. Criminal Detection and Prevention:**
- Police patrol dogs are trained to detect and alert law enforcement officials to criminal activity, suspicious behavior, and potential threats.
- By detecting dangerous criminals or individuals who pose a threat to public safety, patrol dogs help prevent crimes, protect communities, and maintain law and order.

**2. Crime Prevention:**
- Patrol dogs can help deter crime by intimidating potential criminals and making people less likely to commit crimes in areas where they are present.
- Their presence can create a sense of security and deter individuals from committing crimes.

**3. Search and Rescue Operations:**
- Police patrol dogs are trained to assist law enforcement officers in searching for missing persons, lost children, and other individuals who are unable to 

#### Question 2 - No RAG

In [118]:
query_2 = "What abilities must the police patrol dog exhibit?"

full_response_text = bot.generate_response_without_rag(query_2)

print(query_2, '\n')
print(full_response_text)

What abilities must the police patrol dog exhibit? 

The abilities the police patrol dog must exhibit can be categorized into three main areas:

**1. Physical Abilities:**

* **Agility:** The dog must be able to move quickly and efficiently, including running, jumping, and maneuvering through tight spaces.
* **Strength:** The dog should be able to carry and handle suspects or criminals without compromising their safety.
* **Endurance:** Police patrol dogs often work long shifts, so they need to be able to maintain their energy levels and physical fitness.
* **Balance:** The dog must be able to balance on one leg while performing tasks, such as searching or tracking.

**2. Behavioral Abilities:**

* **Intelligence:** Police dogs need to be intelligent and able to learn new tasks and commands quickly.
* **Alertness:** They must be alert to their surroundings and pick up on even the slightest signs of danger.
* **Attention:** The dog must be able to focus and maintain attention during tra

#### Question 3 - No RAG

In [120]:
full_response_text = bot.generate_response_without_rag(query_3)

print(query_3, '\n')
print(full_response_text)

What is the A K-9 team comprised of? 

The context does not provide any information about the A K-9 team, so I cannot answer this question from the provided context.


In [121]:
full_response_text = bot.generate_response_without_rag(query_4)

print(query_4, '\n')
print(full_response_text)

Describe the basic training for K-9 patrol team officers. 

Sure, here's a description of the basic training for K-9 patrol team officers:

**Phase 1: Foundation Training**

* **Basic Obedience Training:**
    * Establish obedience commands such as sit, stay, come, heel, and down.
    * Focus on commands in various locations and with distractions.
    * Gradually introduce more complex commands and scenarios.
* **Animal Handling and Control:**
    * Learn proper handling techniques for dogs of different breeds and temperaments.
    * Develop methods for controlling aggressive or reactive dogs.
    * Understand basic animal welfare principles.
* **Vehicle Control:**
    * Get familiar with driving procedures and traffic laws.
    * Practice handling and controlling a K-9 police vehicle.
    * Understand emergency procedures in the vehicle.

**Phase 2: Specialized Training**

* **Scent Work Training:**
    * Teach dogs to detect and identify specific scents associated with criminal activ

In [ ]:
full_response_text = bot.generate_response_without_rag(query_5)

print(query_5, '\n')
print(full_response_text)

## Conclusion:

The chat bot performs much better with RAG access to the PDF document. With RAG the chat bot give shorter, more concise specific and accurate responses. Without RAG the chat bot gives a longer more imprecise response as it attempts to answer the question generally without specific reference the PDF